In [1]:
import numpy as np
import random
from scipy.optimize import minimize

Для удобства поле можно считать массивом из 4 элементов (0 - обычные клетки, -1 - мина, 1 - бонус)

В независимости от стратегии игрока (случайно выбирает клетки или по какому-то паттерну) можно считать все 12 возможных полей (4 ячейки для размещения бонуса и 3 оставшиеся для размещения мины) равноверятными

Также будем считать, что человек еще до начала игры случайно выбирает, сколько клеток будет открывать (то есть человек может хотеть открыть 3 клетки, но мина выпадет самой первой)

Логично сделать $1 < k_1 < k_2 < k_3$ и $\forall b_i > 1$

Очевидно, что на уровне с бонусом не может быть никаких стратегий, человек выбирает ячейку случайно, поэтому на финальное матожидание будет влиять среднее значение коэффициентов b, значит коэффициенты b можно будет выбрать любые, главное чтобы у них было определенное среднее (логично выбрать коэффициенты b с большой дисперсией, чтобы у человека разыгрался азарт)

$$E_b = \frac{b_1 + b_2 + b_3 + b_4}{4}$$

Можно расписать все возможные варианты выйгрышей:

| Поле   | 1 клетка       | 2 клетки       | 3 клетки       |
|-----------------------|--------------|--------------|--------------|
| [-1, 1, 0, 0]         | 0            | 0            | 0            |
| [-1, 0, 1, 0]         | 0            | 0            | 0            |
| [-1, 0, 0, 1]         | 0            | 0            | 0            |
| [1, -1, 0, 0]         | $E_bk_1$     | 0            | 0            |
| [0, -1, 1, 0]         | $k_1$        | 0            | 0            |
| [0, -1, 0, 1]         | $k_1$        | 0            | 0            |
| [1, 0, -1, 0]         | $E_bk_1$     | $E_bk_2$     | 0            |
| [0, 1, -1, 0]         | $k_1$        | $E_bk_2$     | 0            |
| [0, 0, -1, 1]         | $k_1$        | $k_2$        | 0            |
| [1, 0, 0, -1]         | $E_bk_1$     | $E_bk_2$     | $E_bk_3$     |
| [0, 1, 0, -1]         | $k_1$        | $E_bk_2$     | $E_bk_3$     |
| [0, 0, 1, -1]         | $k_1$        | $k_2$        | $E_bk_3$     |

В таком случае матожидание будет:

$$E = \frac{6 k_1 + 3E_bk_1 + 2 k_2 + 4E_bk_2 + 3 E_bk_3}{36} = \frac{(6 + 3E_b)k_1 + (2 + 4E_b) k_2 + 3 E_bk_3}{36}$$

Также можно посчитать матожидание для стратегий "открывать одну клетку" и "открывать две клетки":

$$ E_1 = \frac{6k_1 + 3E_bk_1}{12} = \frac{(2 + E_b)k_1}{4}, \quad E_2 = \frac{6k_1 + 3E_bk_1 + 2k_2 + 4E_bk_2}{24} = \frac{(6 + 3E_b)k_1 + (2 + 4E_b)k_2}{24}$$

Сравним формулу с численным подсчетом матожидания (через Монте-Карло)

In [2]:
def one_game_simulation(k, b):
    '''
        Симуляция одной случайной игры
    '''
    field = [0, 0, 0, 0]

    bonus_idx, mine_idx = random.sample(range(4), 2)
    field[bonus_idx], field[mine_idx] = 1, -1
    number_steps = random.randint(1, 3)

    k_cur, b_cur = 1, 1
    for i in range(number_steps):
        if field[i] == -1:
            return 0
        if field[i] == 1:
            b_cur = b[random.randint(0, 3)]
        k_cur = k[i]
    return k_cur * b_cur


def monte_carlo(k, b, N=100000):
    '''
        Подсчет матожидания через метод Монте-Карло
    '''
    sum_ = 0
    for _ in range(N):
        sum_ += one_game_simulation(k, b)
    return sum_ / N


def anal_E(k, b):
    '''
        Аналитическая формула матожидания
    '''
    mean_b = sum(b) / 4
    return (6 * k[0] + 2 * k[1] + 3 * mean_b * k[0] + 4 * mean_b * k[1] + 3 * mean_b * k[2]) / 36


k = [1.2, 2.5, 4.0]
b = [10, 100, 1000, 10000]
print(f'Симуляция (Монте-Карло): {monte_carlo(k, b)}, Аналитическое значение: {anal_E(k, b)}')

Симуляция (Монте-Карло): 2002.6695679995835, Аналитическое значение: 1975.4499999999998


$$E = 0.9 = \frac{(6 + 3E_b)k_1 + (2 + 4E_b) k_2 + 3 E_bk_3}{36} > \frac{(6 + 3E_b) + (2 + 4E_b) + 3 E_b}{36} = \frac{8 + 10E_b}{36} \Rightarrow E_b < 2.44$$

Можно рассмотреть более срогое ограничение $E_1 \leq 0.9$ (т.к. матожидание должно быть одинаковое при любой стратегии)

$$ E_1 = \frac{(2+E_b)k_1}{4} \leq 0.9 \Rightarrow E_b \leq \frac{3.6}{k_1} - 2 < 3.6-2 = 1.6$$

Можно взять $b = [1.2, 1.3, 1.5, 2.0], \; E_b = 1.5$. Тогда

$$E = 0.9 = \frac{10.5k_1 + 8 k_2 + 4.5k_3}{36} = \frac{21k_1 + 16 k_2 + 9k_3}{72}$$

Зададим дополнительные ограничения на параметры $k$, чтобы $E_1 \leq 0.9$ и $E_2 \leq 0.9$:

$$ E_1 = \frac{(2 + E_b)k_1}{4} = \frac{3.5}{4} k_1 \leq 0.9 \Rightarrow k_1 \leq \frac{36}{35}$$

$$E_2 = \frac{(6 + 3E_b)k_1 + (2 + 4E_b)k_2}{24} = \frac{10.5k_1 + 8k_2}{24}  \leq 0.9 \Rightarrow 21k_1 + 16k_2 \leq 43.2$$

Осталось решить следующую задачу:
$$ \begin{cases}
21k_1 + 16k_2 + 9k_3 = 64.8 \\
1 < k_1 \leq \frac{36}{35} \\
k_2 > k_1 \\
21k_1 + 16k_2 \leq 43.2 \\
k_3 > k_2 \\
\end{cases} $$

In [3]:
def objective(k):
    return abs(21 * k[0] + 16 * k[1] + 9 * k[2] - 64.8)

def constraint(k):
    return [
        k[0] - 1,                     # k_1 > 1
        36/35 - k[0],                 # k_1 <= 36/35
        k[1] - k[0],                  # k_2 > k_1
        43.2 - 21 * k[0] - 16 * k[1], # 21k_1 + 16k_2 <= 43.2
        k[2] - k[1]                   # k_3 > k_2
    ]

k_initial = [1.1, 1.2, 1.3]

result = minimize(
    objective,
    k_initial,
    constraints={
        'type': 'ineq',
        'fun': constraint
    }
)

k_optimal = result.x
print("Найденные значения k:", k_optimal)
print("Ошибка:", objective(k_optimal))

Найденные значения k: [1.02857143 1.22131966 2.62876505]
Ошибка: 2.29654517625022e-09


Таким образом, подходят значения $k_1 = 1.03, \; k_2 = 1.22, \; k_3 = 2.63, b = [1.2, 1.3, 1.5, 2.0]$. Проверим аналитиески и с помощью Монте-Карло

In [4]:
k = [1.03, 1.22, 2.63]
b = [1.2, 1.3, 1.5, 2.0]
print(f'Симуляция (Монте-Карло): {monte_carlo(k, b)}, Аналитическое значение: {anal_E(k, b)}')

Симуляция (Монте-Карло): 0.8983833200000498, Аналитическое значение: 0.9002777777777777


Это не единственные подходящие значения, их можно варьировать (при уменьшении значения $E_b$ будут увеличиваться значения коэффициентов $k$)

Также можно проверить, что при двух других стратегиях математическое ожидание будет не больше 0.9

In [5]:
k = [1.03, 1.22, 2.63]
b = [1.2, 1.3, 1.5, 2.0]

def one_step(k, b):
    mean_b = sum(b) / 4
    return (2 + mean_b) * k[0] / 4

def two_step(k, b):
    mean_b = sum(b) / 4
    return ((6 + 3 * mean_b) * k[0] + (2 + 4 * mean_b) * k[1]) / 24

print(f'Матожидание одношаговой стратегии {one_step(k, b):.3f}, матожидание двушаговой стратегии: {two_step(k, b):.3f}\n')

Матожидание одношаговой стратегии 0.901, матожидание двушаговой стратегии: 0.857



Можно обощить задачу таким образом (пусть необходимое матожидание будет $\varepsilon = 0.9$):

Необходимо выбрать $1 < E_b < 4\varepsilon - 2$, а затем решить следующую задачу:

$$ \begin{cases}
(6 + 3E_b)k_1 + (2 + 4E_b) k_2 + 3 E_bk_3 = 36 \varepsilon \\
1 < k_1 \leq \frac{4 \varepsilon}{2+E_b} \\
k_2 > k_1 \\
(6 + 3E_b)k_1 + (2 + 4E_b)k_2 \leq 24 \varepsilon \\
k_3 > k_2 \\
\end{cases} $$



In [6]:
def generate_coefs(b_mean, expected_mean=0.9):

    # Один из возможных способов задать коэффициенты b с заданным матожиданием
    delta_b = b_mean - 1
    b = [
        b_mean - delta_b / 2,
        b_mean - delta_b / 3,
        b_mean + delta_b / 3,
        b_mean + delta_b / 2
    ]

    def objective(k):
        return abs((6 + 3 * b_mean) * k[0] + (2 + 4 * b_mean) * k[1] + 3 * b_mean * k[2] - 36 * expected_mean)

    def constraint(k):
        return [
            k[0] - 1,
            expected_mean - (2 + b_mean) / 4 * k[0],
            k[1] - k[0],
            24 * expected_mean - ((6 + 3 * b_mean) * k[0] + (2 + 4 * b_mean) * k[1]),
            k[2] - k[1]
        ]

    k_initial = [1.01, 1.02, 1.03]

    result = minimize(
        objective,
        k_initial,
        constraints={
            'type': 'ineq',
            'fun': constraint
        }
    )

    return result.x, np.array(b)

In [7]:
for i in range(1, 7):
    b_mean = 1 + i / 10
    k, b = generate_coefs(b_mean)
    print(f'b_mean = {b_mean}, k = {k.round(3)}, b = {b.round(3)}')
    print(f'Монте-Карло: {monte_carlo(k, b):.3f}')
    print(f'Аналитическое решение: {anal_E(k, b):.3f}')
    print(f'Матожидание одношаговой стратегии {one_step(k, b):.3f}')
    print(f'Матожидание двушаговой стратегии: {two_step(k, b):.3f}\n')

b_mean = 1.1, k = [1.129 1.547 3.635], b = [1.05  1.067 1.133 1.15 ]
Монте-Карло: 0.901
Аналитическое решение: 0.900
Матожидание одношаговой стратегии 0.875
Матожидание двушаговой стратегии: 0.850

b_mean = 1.2, k = [1.087 1.402 3.452], b = [1.1   1.133 1.267 1.3  ]
Монте-Карло: 0.900
Аналитическое решение: 0.900
Матожидание одношаговой стратегии 0.870
Матожидание двушаговой стратегии: 0.832

b_mean = 1.3, k = [1.056 1.292 3.242], b = [1.15 1.2  1.4  1.45]
Монте-Карло: 0.898
Аналитическое решение: 0.900
Матожидание одношаговой стратегии 0.871
Матожидание двушаговой стратегии: 0.823

b_mean = 1.4, k = [1.033 1.21  3.016], b = [1.2   1.267 1.533 1.6  ]
Монте-Карло: 0.902
Аналитическое решение: 0.900
Матожидание одношаговой стратегии 0.878
Матожидание двушаговой стратегии: 0.822

b_mean = 1.5, k = [1.017 1.149 2.784], b = [1.25  1.333 1.667 1.75 ]
Монте-Карло: 0.903
Аналитическое решение: 0.900
Матожидание одношаговой стратегии 0.890
Матожидание двушаговой стратегии: 0.828

b_mean = 1.6, 

В файле `game.html` можно попробовать самому поиграть в игру с необходимыми коэффициентами